# Migration script

This script is useful to move from v1 to v2.

- Save the database and make a copy of the files just in case
- Make sure the table DCE contains all the fields (see create_table.ipynb)
- Make sure the files are stored in the directory filled in the config file
- Run this script to add the file sizes and set the processing state of each DCE
- You can delete the table `files`

In [1]:
import os

import psycopg2

from scraper_place.config import CONFIG_DATABASE, CONFIG_FILE_STORAGE, STATE_FETCH_OK, build_internal_filepath

In [2]:
# Open connection
connection = psycopg2.connect(
    dbname=CONFIG_DATABASE['name'],
    user=CONFIG_DATABASE['username'],
    password=CONFIG_DATABASE['password'],
)
cursor = connection.cursor()

In [3]:
cursor.execute(
    """
    SELECT annonce_id, org_acronym, intitule,
        filename_reglement, filename_complement, filename_avis, filename_dce
    FROM dce
    ;"""
)
dce_data_list = cursor.fetchall()

In [4]:
for dce_data in dce_data_list:
    (
        annonce_id, org_acronym, intitule,
        filename_reglement, filename_complement, filename_avis, filename_dce
    ) = dce_data

    file_types = ['reglement', 'complement', 'avis', 'dce']
    filenames = [filename_reglement, filename_complement, filename_avis, filename_dce]
    for file_type, filename in zip(file_types, filenames):
        if filename:
            internal_filepath = build_internal_filepath(annonce_id, org_acronym, filename, file_type)
            file_size = os.path.getsize(internal_filepath)
            
            psql_request_template = """
                UPDATE dce
                SET file_size_{} = %s
                WHERE annonce_id = %s AND org_acronym = %s
                ;""".format(file_type)
            cursor.execute(
                psql_request_template,
                (file_size, annonce_id, org_acronym)
            )
    
    cursor.execute(
        """
        UPDATE dce
        SET state = %s
        WHERE annonce_id = %s AND org_acronym = %s
        ;""",
        (STATE_FETCH_OK, annonce_id, org_acronym)
    )

    connection.commit()

In [5]:
cursor.close()
connection.close()